In [26]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster 
from pyproj import Porj,transform
#패키지 import

You should consider upgrading via the 'C:\Users\gymdo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'pyproj'

In [16]:
df = pd.read_csv('./data/편의시설/부산광역시_종합병원 현황_20211231.csv'
, encoding='UTF-8') #부산 경로당 현황 읽어오기 
hos_sub_df = df[df['도로명주소'].str.contains('기장군')]#관리기관명이 기장군인 것만 자름
print(hos_sub_df[['위도','경도','의료기관명']])


           위도          경도      의료기관명
27  35.321839  129.244334  동남권원자력의학원


In [24]:
df = pd.read_csv('./data/편의시설/fulldata_01_01_06_P_약국.csv'
, encoding='ANSI') #부산 경로당 현황 읽어오기 
df.head(3)
pha_sub_df = df[df['소재지전체주소'].str.contains('기장군')]#관리기관명이 기장군인 것만 자름
print(df[['좌표정보(x)','좌표정보(y)']])


             좌표정보(x)        좌표정보(y)
0                NaN            NaN
1      193786.830157  185954.519436
2      194075.244057  184984.418601
3      193146.400498  186088.106546
4      194270.107173  184985.278274
...              ...            ...
65228  198055.210073  451729.379050
65229  244684.789281  345008.277824
65230  286530.401496  425229.266367
65231  169654.615993  407951.221769
65232  217889.983260  464525.772770

[65233 rows x 2 columns]


C:\Users\gymdo\AppData\Local\Temp\ipykernel_16036\3074028986.py:1: DtypeWarning: Columns (5,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/편의시설/fulldata_01_01_06_P_약국.csv'


In [17]:
kizang_pos= [35.23876594878238, 129.21575390899562] ## 기장읍 사무소
m = folium.Map(location=kizang_pos, zoom_start=17,width=750, 
               height=500)#지도 17의 줌으로 생성
coords = hos_sub_df[['위도','경도','의료기관명']]#자료에서 위도, 경도, 경로당명 가져와서
hos_marker_cluster = MarkerCluster().add_to(m)#MarkerCluster을 지도에 추가하고
for lat,long,tool in zip(coords['위도'],coords['경도'],coords['의료기관명']):#lat, long, tool을 이터레이터로 만들어서
    folium.Marker([lat,long],icon=folium.Icon(color='green'),tooltip=tool).add_to(hos_marker_cluster)#초록색 마커로 마우스 갖다대면 이름 보이게 표시
m
